In [1]:
%pwd

'C:\\NN-P4\\python-notebooks'

###Imports and Read CSV

In [2]:
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from datetime import datetime
from keras.callbacks import History # to get train accuracy
history = History()

In [7]:
#nPrintML creates a copy of dataset in pkl picles format. Convert to CSV 1st.
# Read the PKL
df_train = pd.read_pickle('../datasets/nprint-raw/stream-video-half-c10/X.pkl')
df_test = pd.read_pickle('../datasets/nprint-raw/stream-video-half-c10/X_val.pkl')
label_train = pd.read_pickle('../datasets/nprint-raw/stream-video-half-c10/y.pkl')
label_test = pd.read_pickle('../datasets/nprint-raw/stream-video-half-c10/y_val.pkl')
# Save a copy as CSV
df_train.to_csv('../datasets/nprint-raw/stream-video-half-c10/X.csv')
df_test.to_csv('../datasets/nprint-raw/stream-video-half-c10/X_val.csv')
label_train.to_csv('../datasets/nprint-raw/stream-video-half-c10/y.csv')
label_test.to_csv('../datasets/nprint-raw/stream-video-half-c10/y_val.csv')
# Read CSV
df_train = pd.read_csv('../datasets/nprint-raw/stream-video-half-c10/X.csv')
df_test = pd.read_csv('../datasets/nprint-raw/stream-video-half-c10/X_val.csv')
label_train = pd.read_csv('../datasets/nprint-raw/stream-video-half-c10/y.csv')
label_test = pd.read_csv('../datasets/nprint-raw/stream-video-half-c10/y_val.csv')
#Drop colum with pcap sequence number
df_train.drop('pcap', axis=1, inplace=True)
df_test.drop('pcap', axis=1, inplace=True)
label_train.drop('pcap', axis=1, inplace=True)
label_test.drop('pcap', axis=1, inplace=True)
#Rename Unnamed Collumn
label_train.rename(columns={'0': 'label'}, inplace=True)
label_test.rename(columns={'0': 'label'}, inplace=True)
#Read file with ordered list of attributes ranked by the autoML feature importance algorithm
features_rankings_df = pd.read_csv('../datasets/nprint-raw/stream-video-half-c10/feature-importance-half-c10-f-r.csv')
features_rankings_df.rename(columns={'0': 'label'}, inplace=True)

In [ ]:
display(df_train)
display(df_test)
display(label_train)
display(label_test)
print(label_train["label"].value_counts())
print(label_test["label"].value_counts())
print(f"df_train shape: {df_train.shape} \n columns: {df_train.columns.values}")
print(f"df_test shape: {df_test.shape} \n columns: {df_test.columns.values}")
display(features_rankings_df.head(40))

In [9]:
#make a list with only the best features

feature_list = features_rankings_df['Unnamed: 0'].tolist()
feature_list = feature_list[:32]
print(feature_list)

#select a sub-dataframe with only the 32 top attrs rate by the AUTOGLUON feature importance algorithm
df_train = df_train[feature_list]
df_test = df_test[feature_list]

display(df_train)
display(df_test)

['pkt_3_ipv4_src_23', 'pkt_0_ipv4_src_25', 'pkt_9_ipv4_dst_5', 'pkt_2_rts', 'pkt_9_ipv4_dst_6', 'pkt_9_ipv4_dst_0', 'pkt_4_rts', 'pkt_3_rts', 'pkt_7_ipv4_dst_27', 'pkt_6_tcp_opt_60', 'pkt_6_ipv4_dst_4', 'pkt_0_ipv4_src_18', 'pkt_7_rts', 'pkt_9_ipv4_src_28', 'pkt_4_tcp_opt_60', 'pkt_0_tcp_opt_60', 'pkt_9_ipv4_dst_4', 'pkt_8_rts', 'pkt_4_ipv4_dst_1', 'pkt_8_ipv4_src_28', 'pkt_9_ipv4_dst_3', 'pkt_5_rts', 'pkt_7_ipv4_dst_11', 'pkt_1_rts', 'pkt_9_ipv4_src_21', 'pkt_7_ipv4_dst_9', 'pkt_9_ipv4_dst_13', 'pkt_4_ipv4_dst_12', 'pkt_7_ipv4_dst_0', 'pkt_6_rts', 'pkt_2_ipv4_dst_19', 'pkt_3_ipv4_dst_7']


,pkt_3_ipv4_src_23,pkt_0_ipv4_src_25,pkt_9_ipv4_dst_5,pkt_2_rts,pkt_9_ipv4_dst_6,pkt_9_ipv4_dst_0,pkt_4_rts,pkt_3_rts,pkt_7_ipv4_dst_27,pkt_6_tcp_opt_60,...,pkt_7_ipv4_dst_11,pkt_1_rts,pkt_9_ipv4_src_21,pkt_7_ipv4_dst_9,pkt_9_ipv4_dst_13,pkt_4_ipv4_dst_12,pkt_7_ipv4_dst_0,pkt_6_rts,pkt_2_ipv4_dst_19,pkt_3_ipv4_dst_7
0,1,0,0,1362319,0,0,71,132,0,0,...,1,296,0,0,0,1,0,69,0,0
1,1,0,1,285,1,0,926,104160,0,1,...,0,4390256,0,0,0,0,0,634,0,0
2,1,0,1,326,0,0,219,205,1,1,...,0,1352462,0,1,0,1,0,121,1,0
3,1,0,0,33,0,1,14186,20,0,1,...,1,43,1,1,0,0,1,38,1,1
4,1,1,0,1190046,0,1,59,116,1,1,...,1,244,0,1,0,1,1,55,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6592,1,0,0,241610,1,1,7998,621382,0,1,...,1,11881,0,0,1,1,0,44405,1,0
6593,1,0,1,214,1,0,142,1531520,0,0,...,0,320,0,0,0,0,0,78,0,0
6594,1,0,1,155,1,1,445725,1395299,1,0,...,1,162,0,0,0,1,0,505802,1,0
6595,1,0,1,27,1,0,24,24,0,1,...,0,150058,0,0,0,0,0,20,1,0


,pkt_3_ipv4_src_23,pkt_0_ipv4_src_25,pkt_9_ipv4_dst_5,pkt_2_rts,pkt_9_ipv4_dst_6,pkt_9_ipv4_dst_0,pkt_4_rts,pkt_3_rts,pkt_7_ipv4_dst_27,pkt_6_tcp_opt_60,...,pkt_7_ipv4_dst_11,pkt_1_rts,pkt_9_ipv4_src_21,pkt_7_ipv4_dst_9,pkt_9_ipv4_dst_13,pkt_4_ipv4_dst_12,pkt_7_ipv4_dst_0,pkt_6_rts,pkt_2_ipv4_dst_19,pkt_3_ipv4_dst_7
0,1,0,1,151,1,0,53,1431882,0,0,...,0,370,0,0,0,0,0,314,1,0
1,1,0,0,2076744,0,0,3786926,119549565,1,0,...,1,1197,0,0,0,1,0,31323367,0,0
2,1,0,0,967187,0,0,150,196629,0,1,...,1,7963,0,0,1,1,0,58844,0,0
3,1,0,1,94780,1,0,319,274,1,1,...,0,272,0,0,0,0,0,704,1,0
4,1,0,1,40,1,0,7267,36,1,1,...,1,72,1,0,1,0,1,35,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,0,1,0,421237,0,0,47848,121529,0,0,...,1,119321,1,0,0,1,0,183,0,0
729,1,1,0,72,0,0,3044,12488,0,1,...,1,3547,0,0,0,1,0,56,1,0
730,1,0,0,1069169,1,1,170952,104,0,0,...,0,284,0,1,0,1,0,73,0,1
731,1,0,1,148,0,0,1178443,1453628,0,0,...,1,167,0,0,0,1,0,277150,1,0


In [10]:
# Model Parameters
number_of_attributes = df_train.shape[1]
hidden_layer_nodes = 256
output_layer_nodes = 4

# Traing Parameters
EPOCHS = 500
BATCH_SIZE = 500

# Run the NN Training multiple times to get a nice mean score of accuracy results
NUMBER_OF_NN_REPETITIONS = 10

def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(hidden_layer_nodes, activation='relu'),
    tf.keras.layers.Dense(output_layer_nodes)
  ])
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [11]:
# Convert int values to TF expect values (float)
df_train = np.asarray(df_train).astype(np.float32)
df_test = np.asarray(df_test).astype(np.float32)
# Converts pandas dataframe to tensorflow object
df_train = tf.convert_to_tensor(df_train)
# Set the 
normalizer = tf.keras.layers.Normalization(axis=-1)
# Normalize the data
normalizer.adapt(df_train)
#normalizer.adapt(numeric_features_test)

In [12]:
# Execute Trainning and Testing
#model = get_basic_model()
train_accuracy = []
test_accuracy = []

for i in range(NUMBER_OF_NN_REPETITIONS):
  model = get_basic_model()
  history = model.fit(df_train, label_train, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE, callbacks=[history])
  one_train_accuracy = history.history['accuracy']

  test_loss, one_test_accuracy =  model.evaluate(df_test,  label_test, verbose=1, batch_size=BATCH_SIZE)
  train_accuracy.append(one_train_accuracy[-1])
  test_accuracy.append(one_test_accuracy)
  print(f"{i+1}: train_accuracy:{train_accuracy} test_accuracy:{test_accuracy}")
  tf_predictions_probabilities = model.predict(df_test)
  #tf_loss, tf_acc =  model.evaluate(numeric_features_test,  label_test, verbose=2, batch_size=BATCH_SIZE)

  #Confusion Matrix
  tf_predictions = []
  for i,x in enumerate(tf_predictions_probabilities):
    #print("i:",i,"x_max:",x.max(),"x:",x)
    j_max = x.argmax()
    tf_predictions.append(j_max)
  conf_m = tf.math.confusion_matrix(label_test,tf_predictions)
  print(conf_m)

#After n repetitions of training and testing, calculate statistics
train_accuracy_mean = sum(train_accuracy) / len(train_accuracy)
train_max = max(train_accuracy)
train_min = min(train_accuracy)
train_std = np.std(train_accuracy)
test_accuracy_mean = sum(test_accuracy) / len(test_accuracy)
test_max = max(test_accuracy)
test_min = min(test_accuracy)
test_std = np.std(test_accuracy)

print(f'accuracies_train: {train_accuracy}, accuracies_test: {test_accuracy}')
print(f'train_accuracy_mean: {train_accuracy_mean*100:.2f}, train_max: {train_max*100:.2f}, train_min: {train_min*100:.2f}, train_std: {train_std*100:.2f} \n test_accuracy_mean: {test_accuracy_mean*100:.2f}, test_max: {test_max*100:.2f}, test_min: {test_min*100:.2f}, test_std: {test_std*100:.2f}')



2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9540 - loss: 0.1540 
1: train_accuracy:[0.9828709959983826] test_accuracy:[0.9549795389175415]
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
tf.Tensor(
[[ 61   4   2   0]
 [  6 214   3   3]
 [  5   0 108   0]
 [  0   8   2 317]], shape=(4, 4), dtype=int32)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9540 - loss: 0.1671 
2: train_accuracy:[0.9828709959983826, 0.9816583395004272] test_accuracy:[0.9549795389175415, 0.9549795389175415]
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
tf.Tensor(
[[ 61   4   2   0]
 [  6 211   3   6]
 [  5   0 108   0]
 [  0   5   2 320]], shape=(4, 4), dtype=int32)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9508 - loss: 0.1845 
3: train_accuracy:[0.9828709959983826, 0.9816583395004272, 0.9836289286613464] test_accuracy:[0.9549795389175415, 0.9549795389175415, 0.9522510170936584]
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
tf.Tensor(
[[ 61   4   2   0]
 [  7 212   4   3]
 [  4   0 109   0]
 [  0   8   3 316]], s

In [12]:
# Date for report file
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
print(dt_string)
# Report file content
report = {
    "datetime": dt_string,
    "training_dataset_shape":  str(df_train.shape),
    "test_dataset_shape": str(df_test.shape),
    "number_of_attributes": number_of_attributes,
    "n_epoch":EPOCHS,
    "accuracy_train": train_accuracy_mean,
    "train_max": train_max,
    "train_min": train_min,
    "train_std": train_std,
    "accuracy_test": test_accuracy_mean,
    "test_max": test_max,
    "test_min": test_min,
    "test_std": test_std,
    "accuracies_train": train_accuracy,
    "accuracies_test": test_accuracy
}

# Writes into the file
os.chdir('../tf-model-testing')
print(os.getcwd())

title_parameters_save = f"nn-nprint-app-iden-model-evaluation-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

title_parameters_save = f"nn-nprint-app-iden-model-evaluation-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}-{dt_string}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

12-06-2024-16-27-20
C:\NN-P4\tf-model-testing
